In [39]:
# Importing necessary libraries
import pandas as pd 
import seaborn as sns 
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 
import missingno as msno
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.metrics import accuracy_score, log_loss, cohen_kappa_score, f1_score
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.multioutput import MultiOutputClassifier

In [40]:
train_data = pd.read_csv('data/Training.csv')

test_data = pd.read_csv('data/Testing.csv')

train_data.head(10) # looking at the first few rows of data 
total_columns = len(train_data.columns)
total_rows = len(train_data['prognosis'])
prognoses = train_data['prognosis'].unique().tolist()

print(f"The training dataset has a total of {total_rows} rows and {total_columns} columns. This means that there are {total_columns - 1} predictor variables. \nwithin the prognosis column, there are {len(prognoses)} diseases included.")

The training dataset has a total of 4920 rows and 134 columns. This means that there are 133 predictor variables. 
within the prognosis column, there are 41 diseases included.


In [41]:
# drop column with no value
train_data = train_data.drop('Unnamed: 133', axis=1)

In [42]:
# Encode the categorical target for correlation analysis
le = LabelEncoder()
train_data['disease_encoded'] = le.fit_transform(train_data['prognosis'])

In [43]:
train_data['disease_encoded'].unique()

array([15,  4, 16,  9, 14, 33,  1, 12, 17,  6, 23, 30,  7, 32, 28, 29,  8,
       11, 37, 40, 19, 20, 21, 22,  3, 36, 10, 34, 13, 18, 39, 26, 24, 25,
       31,  5,  0,  2, 38, 35, 27])

In [45]:
train_data['disease_encoded'] = train_data['disease_encoded'].astype(float)


In [47]:
cat_cols = ['prognosis']

In [48]:
# drop column with no value
train_data = train_data.drop('Unnamed: 133', axis=1)

KeyError: "['Unnamed: 133'] not found in axis"

In [ ]:
train_data.shape

(4920, 134)

In [ ]:
pipeline_A = Pipeline([
    ('classifier', DecisionTreeClassifier())
])
pipeline_A

Pipeline(steps=[('classifier', DecisionTreeClassifier())])

In [ ]:
pipeline_B = Pipeline([
    ('classifier', RandomForestClassifier())
])
pipeline_B

Pipeline(steps=[('classifier', RandomForestClassifier())])

In [ ]:
pipeline_C = Pipeline([
    ('classifier', XGBClassifier())
])
pipeline_C

Pipeline(steps=[('classifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

In [49]:
pipeline_D = Pipeline([
    ('classifier', OneVsRestClassifier(LogisticRegression(max_iter=1000)))
])
pipeline_D

Pipeline(steps=[('classifier',
                 OneVsRestClassifier(estimator=LogisticRegression(max_iter=1000)))])

In [50]:
pipeline_E = Pipeline([
    ('classifier', SVC())
])
pipeline_E

Pipeline(steps=[('classifier', SVC())])

In [51]:
X_train = train_data.drop(columns=['prognosis'])

Y_train = train_data['prognosis']

X_test = test_data.drop(columns=['prognosis'])

Y_test = test_data['prognosis']

scoring = ['neg_log_loss', 'roc_auc', 'f1', 'accuracy', 'precision', 'recall']

In [52]:
# Custom target transformer (Label Encoding for classification task)
class CustomTargetTransformer:
    def fit(self, y):
        # Fit the LabelEncoder to the target variable
        self.encoder = LabelEncoder()
        self.encoder.fit(y)
        return self

    def transform(self, y):
        # Transform the target variable to encoded values
        return self.encoder.transform(y)

    def inverse_transform(self, y):
        # Inverse transform to get the original target variable back
        return self.encoder.inverse_transform(y)

Best for: When you want to treat all classes equally, regardless of their size.

Explanation: The macro average computes the metric (precision, recall, or F1) for each class independently, and then averages these scores. This is useful when the classes are of equal importance.

Use Case: If you want to ensure that your model performs equally well across all classes, without regard to class distribution, the macro average may be the best choice.

For now we used f1_weighted as the metric.

In [53]:
param_grid_A = {
    'classifier__max_depth': [10, 20, 30],
    'classifier__max_features': ['sqrt', 'log2']
}

grid_search_A = GridSearchCV(
    pipeline_A,
    param_grid_A, cv=5,
    scoring= 'f1_weighted',
    refit= True
)

# Apply the transformation to the target variable (Y_train) outside of the pipeline
target_transformer = CustomTargetTransformer()
target_transformer.fit(Y_train)

# Fit the target transformer on Y_train and transform it
Y_train_transformed = target_transformer.transform(Y_train)

grid_search_A.fit(X_train, Y_train_transformed)
model_A_df = pd.DataFrame(grid_search_A.cv_results_)
model_A_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__max_depth,param_classifier__max_features,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.007667,0.000810,0.003803,0.000422,10,sqrt,"{'classifier__max_depth': 10, 'classifier__max...",0.277785,0.370491,0.292179,0.423374,0.245427,0.321851,0.065323,6
1,0.006783,0.000389,0.003217,0.000183,10,log2,"{'classifier__max_depth': 10, 'classifier__max...",0.327405,0.287281,0.314539,0.539024,0.261789,0.346008,0.099130,5
2,0.007374,0.000645,0.003128,0.000270,20,sqrt,"{'classifier__max_depth': 20, 'classifier__max...",0.618056,0.697764,0.694285,0.588076,0.588076,0.637251,0.049233,4
3,0.007365,0.000444,0.003106,0.000152,20,log2,"{'classifier__max_depth': 20, 'classifier__max...",0.545852,0.710754,0.587975,0.547967,0.860627,0.650635,0.120961,3
4,0.008064,0.000540,0.003197,0.000402,30,sqrt,"{'classifier__max_depth': 30, 'classifier__max...",0.948210,0.927330,0.868522,0.886179,0.878049,0.901658,0.030728,1
5,0.006780,0.000400,0.003203,0.000194,30,log2,"{'classifier__max_depth': 30, 'classifier__max...",0.834552,0.877116,0.819196,0.912195,0.967480,0.882108,0.053736,2


In [54]:
grid_search_A.best_params_

{'classifier__max_depth': 30, 'classifier__max_features': 'sqrt'}

In [55]:
grid_search_A.best_estimator_

Pipeline(steps=[('classifier',
                 DecisionTreeClassifier(max_depth=30, max_features='sqrt'))])

In [56]:
param_grid_B = {
    'classifier__n_estimators': [50, 100],
    'classifier__max_depth': [10, 20],
    'classifier__max_features': ['sqrt', 'log2']
}

grid_search_B = GridSearchCV(
    pipeline_B,
    param_grid_B, cv=5,
    scoring= 'f1_weighted',
    refit=True
)


# Apply the transformation to the target variable (Y_train) outside of the pipeline
target_transformer = CustomTargetTransformer()
target_transformer.fit(Y_train)

# Fit the target transformer on Y_train and transform it
Y_train_transformed = target_transformer.transform(Y_train)

grid_search_B.fit(X_train, Y_train_transformed)
model_B_df = pd.DataFrame(grid_search_B.cv_results_)
model_B_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__max_depth,param_classifier__max_features,param_classifier__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.104937,0.006481,0.005988,0.000660,10,sqrt,50,"{'classifier__max_depth': 10, 'classifier__max...",1.0,0.996939,0.998983,1.0,1.0,0.999185,0.00119,8
1,0.207164,0.005445,0.010350,0.000948,10,sqrt,100,"{'classifier__max_depth': 10, 'classifier__max...",1.0,1.000000,1.000000,1.0,1.0,1.000000,0.00000,1
2,0.097427,0.007152,0.006674,0.000398,10,log2,50,"{'classifier__max_depth': 10, 'classifier__max...",1.0,1.000000,1.000000,1.0,1.0,1.000000,0.00000,1
3,0.176113,0.002454,0.009673,0.000255,10,log2,100,"{'classifier__max_depth': 10, 'classifier__max...",1.0,1.000000,1.000000,1.0,1.0,1.000000,0.00000,1
4,0.130749,0.004809,0.007280,0.000625,20,sqrt,50,"{'classifier__max_depth': 20, 'classifier__max...",1.0,1.000000,1.000000,1.0,1.0,1.000000,0.00000,1
5,0.271141,0.007760,0.011862,0.000917,20,sqrt,100,"{'classifier__max_depth': 20, 'classifier__max...",1.0,1.000000,1.000000,1.0,1.0,1.000000,0.00000,1
6,0.115684,0.005538,0.007935,0.000190,20,log2,50,"{'classifier__max_depth': 20, 'classifier__max...",1.0,1.000000,1.000000,1.0,1.0,1.000000,0.00000,1
7,0.228708,0.003893,0.010946,0.000578,20,log2,100,"{'classifier__max_depth': 20, 'classifier__max...",1.0,1.000000,1.000000,1.0,1.0,1.000000,0.00000,1


In [57]:
grid_search_B.best_params_

{'classifier__max_depth': 10,
 'classifier__max_features': 'sqrt',
 'classifier__n_estimators': 100}

In [58]:
grid_search_B.best_estimator_

Pipeline(steps=[('classifier', RandomForestClassifier(max_depth=10))])

In [59]:
param_grid_C = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [10, 20],
    #'classifier__learning_rate': [0.01, 0.05, 0.1, 0.2],
    #'classifier__subsample': [0.5, 0.7, 1.0],
    #'classifier__colsample_bytree': [0.5, 0.7, 1.0]
}

grid_search_C = GridSearchCV(
    pipeline_C,
    param_grid_C, cv=5,
    scoring= 'f1_weighted',
    refit=True
)

grid_search_C.fit(X_train, Y_train_transformed)
model_C_df = pd.DataFrame(grid_search_C.cv_results_)
model_C_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__max_depth,param_classifier__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.625329,0.130487,0.020118,0.001249,10,50,"{'classifier__max_depth': 10, 'classifier__n_e...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
1,0.979608,0.131932,0.019778,0.002190,10,100,"{'classifier__max_depth': 10, 'classifier__n_e...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
2,1.844944,0.172065,0.022577,0.001498,10,200,"{'classifier__max_depth': 10, 'classifier__n_e...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
3,0.480909,0.037116,0.017124,0.001360,20,50,"{'classifier__max_depth': 20, 'classifier__n_e...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
4,0.794653,0.044026,0.018859,0.002444,20,100,"{'classifier__max_depth': 20, 'classifier__n_e...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
5,1.399210,0.064506,0.020105,0.003265,20,200,"{'classifier__max_depth': 20, 'classifier__n_e...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1


In [60]:
grid_search_C.best_params_

{'classifier__max_depth': 10, 'classifier__n_estimators': 50}

In [61]:
grid_search_C.best_estimator_

Pipeline(steps=[('classifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=10, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=50, n_jobs=None,
                               num_parallel_tree=None,
                               objective='multi:softprob', ...))])

In [62]:
param_grid_D = {
    #'classifier__n_estimators': [50, 100, 200, 500],
    #'classifier__max_depth': [None, 10, 20, 30],
    #'classifier__learning_rate': [0.01, 0.05, 0.1, 0.2],
    #'classifier__subsample': [0.5, 0.7, 1.0],
    #'classifier__colsample_bytree': [0.5, 0.7, 1.0],
    'classifier__estimator__solver': ['liblinear'],
    'classifier__estimator__penalty': ['l1', 'l2'],
    'classifier__estimator__C': [0.01, 0.1, 1]
}

grid_search_D = GridSearchCV(
    pipeline_D,
    param_grid_D, cv=5,
    scoring= {'f1_macro': 'f1_macro', 'roc_auc_ovr': 'roc_auc_ovr'},
    refit='roc_auc_ovr'
)

grid_search_D.fit(X_train, Y_train_transformed)
model_D_df = pd.DataFrame(grid_search_D.cv_results_)
model_D_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__estimator__C,param_classifier__estimator__penalty,param_classifier__estimator__solver,params,split0_test_f1_macro,split1_test_f1_macro,...,std_test_f1_macro,rank_test_f1_macro,split0_test_roc_auc_ovr,split1_test_roc_auc_ovr,split2_test_roc_auc_ovr,split3_test_roc_auc_ovr,split4_test_roc_auc_ovr,mean_test_roc_auc_ovr,std_test_roc_auc_ovr,rank_test_roc_auc_ovr
0,0.367091,0.013926,0.177041,0.019887,0.01,l1,liblinear,"{'classifier__estimator__C': 0.01, 'classifier...",0.002217,0.002217,...,0.000000,6,0.803659,0.803659,0.804268,0.804268,0.803659,0.803902,0.000299,6
1,0.385672,0.009542,0.167632,0.006073,0.01,l2,liblinear,"{'classifier__estimator__C': 0.01, 'classifier...",0.996939,0.997964,...,0.001189,5,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1
2,0.447489,0.029979,0.189638,0.030773,0.10,l1,liblinear,"{'classifier__estimator__C': 0.1, 'classifier_...",1.000000,0.996939,...,0.001224,4,0.999987,0.999994,0.999999,1.000000,1.000000,0.999996,0.000005,5
3,0.412598,0.010629,0.177682,0.017257,0.10,l2,liblinear,"{'classifier__estimator__C': 0.1, 'classifier_...",1.000000,1.000000,...,0.000000,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1
4,0.468388,0.044184,0.187396,0.021349,1.00,l1,liblinear,"{'classifier__estimator__C': 1, 'classifier__e...",1.000000,1.000000,...,0.000000,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1
5,0.444719,0.022222,0.167424,0.014381,1.00,l2,liblinear,"{'classifier__estimator__C': 1, 'classifier__e...",1.000000,1.000000,...,0.000000,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1


In [63]:
grid_search_D.best_params_

{'classifier__estimator__C': 0.01,
 'classifier__estimator__penalty': 'l2',
 'classifier__estimator__solver': 'liblinear'}

In [64]:
grid_search_D.best_estimator_

Pipeline(steps=[('classifier',
                 OneVsRestClassifier(estimator=LogisticRegression(C=0.01,
                                                                  max_iter=1000,
                                                                  solver='liblinear')))])

In [65]:
param_grid_E = {
    'classifier__C': [0.1, 1, 10],
    'classifier__kernel': ['linear', 'rbf'],
    'classifier__gamma': ['scale', 'auto']
}

grid_search_E = GridSearchCV(
    pipeline_E,
    param_grid_E, cv=5,
    scoring= 'f1_weighted',
    refit=True
)

grid_search_E.fit(X_train, Y_train_transformed)
model_E_df = pd.DataFrame(grid_search_E.cv_results_)
model_E_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__C,param_classifier__gamma,param_classifier__kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.088211,0.009203,0.018648,0.000835,0.1,scale,linear,"{'classifier__C': 0.1, 'classifier__gamma': 's...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
1,0.677356,0.026557,0.463524,0.008760,0.1,scale,rbf,"{'classifier__C': 0.1, 'classifier__gamma': 's...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
2,0.082708,0.003241,0.018201,0.000197,0.1,auto,linear,"{'classifier__C': 0.1, 'classifier__gamma': 'a...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
3,0.396983,0.009234,0.462191,0.021145,0.1,auto,rbf,"{'classifier__C': 0.1, 'classifier__gamma': 'a...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
4,0.096424,0.014117,0.019674,0.003069,1.0,scale,linear,"{'classifier__C': 1, 'classifier__gamma': 'sca...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
5,0.197919,0.004769,0.266600,0.012618,1.0,scale,rbf,"{'classifier__C': 1, 'classifier__gamma': 'sca...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
6,0.089655,0.010461,0.016656,0.001014,1.0,auto,linear,"{'classifier__C': 1, 'classifier__gamma': 'aut...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
7,0.131314,0.006032,0.113002,0.006256,1.0,auto,rbf,"{'classifier__C': 1, 'classifier__gamma': 'aut...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
8,0.081221,0.001846,0.016195,0.000816,10.0,scale,linear,"{'classifier__C': 10, 'classifier__gamma': 'sc...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1
9,0.114856,0.005804,0.056990,0.003670,10.0,scale,rbf,"{'classifier__C': 10, 'classifier__gamma': 'sc...",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1


In [66]:
grid_search_E.best_params_

{'classifier__C': 0.1,
 'classifier__gamma': 'scale',
 'classifier__kernel': 'linear'}

In [67]:
grid_search_E.best_estimator_

Pipeline(steps=[('classifier', SVC(C=0.1, kernel='linear'))])

In [68]:
best_model = grid_search_D.best_estimator_
best_model

Pipeline(steps=[('classifier',
                 OneVsRestClassifier(estimator=LogisticRegression(C=0.01,
                                                                  max_iter=1000,
                                                                  solver='liblinear')))])

In [69]:
import pickle

# Save the best model to a pickle file
with open("best_model.pkl", "wb") as file:
    pickle.dump(best_model, file)

print("Best model saved to best_model.pkl")

Best model saved to best_model.pkl


In [72]:
Y_train = pd.get_dummies(Y_train).values

In [73]:
print(Y_train.shape)

(4920, 41)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import shap

# Assuming your dataset is loaded into a DataFrame `df`
# with 'target' as the column for prognosis and symptoms as feature columns

# Initialize and train the RandomForest model
classifier = MultiOutputClassifier(RandomForestClassifier(random_state=42))
classifier.fit(X_train, Y_train)

# Use SHAP to explain the model's predictions
explainer = shap.TreeExplainer(classifier)
shap_values = explainer.shap_values(X_test)

# Display feature importance for each class label
for i, class_name in enumerate(classifier.classes_):
    print(f"\nFeature importances for label: {class_name}")
    shap.summary_plot(shap_values[i].T, X_test, plot_type="bar", max_display=5)

# Optionally display summary plot for more detailed insights
#shap.summary_plot(shap_values, X_test)

In [ ]:
import shap

# Assuming `classifier` is your trained MultiOutputClassifier
# and `X_test` is your test data
shap_values = []
explainer_outputs = []

# Iterate over each target and its respective classifier
for i, estimator in enumerate(classifier.estimators_):
    # Create a SHAP explainer for the underlying RandomForestClassifier
    explainer = shap.TreeExplainer(estimator)
    explainer_outputs.append(explainer)
    
    # Compute SHAP values for the test set
    shap_values_target = explainer.shap_values(X_test)
    shap_values.append(shap_values_target)

# Now `shap_values` is a list where each element corresponds to the SHAP values for one target
